#PYTHON INSTALACJA

In [ ]:

%%shell
pip install hydra-core --upgrade --quiet
pip install unzip --quiet
pip install ipdb -Uqq

     |████████████████████████████████| 154 kB 7.6 MB/s 
     |████████████████████████████████| 79 kB 6.4 MB/s 
     |████████████████████████████████| 117 kB 15.3 MB/s 
     |████████████████████████████████| 761 kB 7.0 MB/s 
     |████████████████████████████████| 1.6 MB 35.4 MB/s 
     |████████████████████████████████| 386 kB 52.2 MB/s 


#PYTHON PAKIETY

In [ ]:
# standardowe pakiety
import os
from os.path import join, isfile
from os import listdir

import numpy as np

import scipy as sp
from scipy import array, newaxis

#from PIL import Image
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
import glob
import ipdb
from __future__ import print_function
from tqdm import tqdm


# Pytorch 
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
import torch.utils.data as data
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torchvision.datasets import ImageFolder

# Hydra
import hydra
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf

# tensorboard
from sklearn.metrics import confusion_matrix
import pandas as pd
from torch.utils.tensorboard import SummaryWriter 
import seaborn as sn


#GENERACJA PLIKÓW HYDRA

Aby załadować checkpoint modelu należy w pliku classification.yaml podać jako parametr `model` jeden z następujących plików:
<br>`cls_model_own_pretrained` - własna implementacja,  
`cls_repo_model_pretrained` - modyfikacja repozytorium

In [ ]:
%%shell
echo "name: classification
target: options
batch_size: 32
num_points: 1000
num_workers: 1
num_epochs: 2
out_folder: 'cls'
model: ''
dataset: 'ModelNet40'
feature_transform: 'store_true'
single_batch_overfit: False" > classification.yaml

echo "name: optimizer
lr: 0.01
betas: [0.9, 0.999]" > optimizer.yaml

echo "name: scheduler
step_size: 20
gamma: 0.5" > scheduler.yaml

In [ ]:
confClass = OmegaConf.load('classification.yaml')
options = instantiate(confClass)

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


#POBIERANIE DANYCH 

In [ ]:
if options.dataset == 'ModelNet40':
    ! wget http://modelnet.cs.princeton.edu/ModelNet40.zip
    ! unzip -qq /content/ModelNet40.zip -d /content/
    dataset_num_classes = 40
    dataset_root = '/content/ModelNet40/'
elif options.dataset == 'ModelNet10':
    ! wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip
    ! unzip -qq /content/ModelNet10.zip -d /content/
    dataset_num_classes = 10
    dataset_root = '/content/ModelNet10/'
else:
    raise ValueError("Wrong model name!")

--2022-12-23 10:08:00--  http://modelnet.cs.princeton.edu/ModelNet40.zip
Resolving modelnet.cs.princeton.edu (modelnet.cs.princeton.edu)... 128.112.136.74
Connecting to modelnet.cs.princeton.edu (modelnet.cs.princeton.edu)|128.112.136.74|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://modelnet.cs.princeton.edu/ModelNet40.zip [following]
--2022-12-23 10:08:00--  https://modelnet.cs.princeton.edu/ModelNet40.zip
Connecting to modelnet.cs.princeton.edu (modelnet.cs.princeton.edu)|128.112.136.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2039180837 (1.9G) [application/zip]
Saving to: ‘ModelNet40.zip’

ModelNet40.zip      100%[===================>]   1.90G  42.7MB/s    in 80s     

2022-12-23 10:09:21 (24.2 MB/s) - ‘ModelNet40.zip’ saved [2039180837/2039180837]



#MODELNET LOADER (WŁASNA IMPLEMENTACJA)

In [ ]:
class ModelNetLoader(data.Dataset):
    def __init__(self, root, randomize_pts=True, load_faces=False, split='train', num_classes=40, numpoints=5000, batch_size=32, transform=None, target_transform=None):
        self.root = root
        self.numpoints = numpoints
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.randomize_pts = randomize_pts
        self.load_faces = load_faces
        self.filelist = [f1 for f1 in glob.glob(self.root + '**/*.off', recursive=True)]
        self.trainlist = [f2 for f2 in self.filelist if "train" in f2]
        self.testlist = [f3 for f3 in self.filelist if "test" in f3]
        if split=='train': self.objlist = self.flist_reader(self.trainlist)
        if split=='test': self.objlist = self.flist_reader(self.testlist)
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        impath, target = self.objlist[index]
        myF = open(impath, 'r')
        firstLine = myF.readline().strip()
        if 'OFF' != firstLine[0:3]:
            raise ValueError('Not a valid OFF header')
        if len(firstLine) > 3:
            n_verts, n_faces, n_edges = tuple([int(s) for s in firstLine[3:].split(' ')])
        else:
            n_verts, n_faces, n_edges = tuple([int(s) for s in myF.readline().strip().split(' ')])
        verts = np.array([[float(s) for s in myF.readline().strip().split(' ')] for i_vert in range(n_verts)])
        choice = np.random.choice(verts.shape[0], self.numpoints, replace=True)
        if self.randomize_pts:
          point_set = verts[choice, :]
        else:
          point_set = verts
        point_set = point_set - np.expand_dims(np.mean(point_set, axis=0), 0) 
        dist = np.max(np.sqrt(np.sum(point_set ** 2, axis=1)), 0)
        point_set = point_set / dist  
        point_set = torch.from_numpy(point_set.astype(np.float32)).float()

        if self.load_faces:
            faces = np.array([[int(s) for s in myF.readline().strip().split(' ')][1:] for i_face in range(n_faces)])
            return point_set, target, faces
        else:
            return point_set, target

    def flist_reader(self, filelist):
        classes_dirs = glob.glob(self.root + "*/", recursive = False)
        classes_list = [i.split('/')[3] for i in classes_dirs]
        classes_nums = np.linspace(0,self.num_classes-1,self.num_classes)
        self.classes_dict = dict(zip(sorted(classes_list), classes_nums.astype(int)))
        self.nums_dict = dict(map(reversed, self.classes_dict.items()))
        objlist = []

        for line in filelist:
            label_start_index = line.strip().find(self.root) + len(self.root)
            label_end_index = line.strip()[label_start_index:].find('/')
            imlabel = line.strip()[label_start_index:label_start_index+label_end_index]
            imlabel = self.classes_dict[imlabel]
            impath = line
            objlist.append( (impath, imlabel) )
        return objlist

    def __len__(self):
      return len(self.objlist)

##DISPLAY SINGLE OBJECT

In [ ]:
def disp(vertices, target, faces, dataset, full):
    
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(projection='3d')
    
    V_np = np.asarray(vertices.cpu())
    if not full:
      ax.scatter(V_np[:, 0], V_np[:, 1], V_np[:, 2])
    else:
      ax.plot_trisurf(V_np[:, 0], V_np[:, 1], V_np[:, 2], triangles=faces);

    plt.axis('off')
    plt.show()
    print('Etykieta klasy: {}'.format(target))
    print(dataset.nums_dict[target])
    print('')



#test for all objects
'''
print(len(mdltest))

for n in range(len(mdltest)):
  xxx = mdltest[n]
  if xxx[0].shape[0] == 0:
    ipdb.set_trace(context=5)
'''



mdltest = ModelNetLoader(root=dataset_root, randomize_pts = False, load_faces=True)
#mdltestR = ModelNetLoader(root=dataset_root, randomize_pts = True, load_faces=True)

ind = random.randint(0, len(mdltest))
xxx = mdltest[ind]
#xxxR = mdltestR[ind]

disp(xxx[0], xxx[1], xxx[2], mdltest, full=False)
disp(xxx[0], xxx[1], xxx[2], mdltest, full=True)
#disp(xxxR[0], xxxR[1], xxxR[2], mdltestR, full=False)


#MODEL

In [ ]:
class STN(nn.Module):
    def __init__(self, kd=True):
        if kd:  self.k=64; self.iden_arr=np.eye(self.k).flatten()
        else:   self.k=3;  self.iden_arr=np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]) 
        super(STN, self).__init__()
        self.conv1 = torch.nn.Conv1d(self.k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, self.k*self.k)
        self.relu = nn.ReLU()

        for x in range(5): 
          string= "self.bn" + str(x+1) + "=nn.BatchNorm1d(" + str(64*2**(x)) + ")"
          exec(string)
        
    def forward(self, x):
        batchsize = x.shape[0]
        #ipdb.set_trace(context=5)
        x = F.relu(self.bn1(self.conv1(x).transpose(2,1)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn5(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024) 
        x = F.relu(self.bn4(self.fc1(x))) 
        x = F.relu(self.bn3(self.fc2(x)))
        x = self.fc3(x)
        iden = torch.from_numpy(self.iden_arr.astype(np.float32)).view(1, self.k*self.k).repeat(batchsize, 1)
        if x.is_cuda:
            iden = iden.to(x.device)
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

class PointNetfeat(nn.Module):
    def __init__(self, global_feat = True, feature_transform = False):
        super(PointNetfeat, self).__init__()
        self.stn = STN(kd=False)
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        if self.feature_transform:
            self.fstn = STN()

    def forward(self, x):
        n_pts = x.shape[2]
        trans = self.stn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2,1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2,1)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans, trans_feat


def feature_transform_regularizer(trans):
    d = trans.size()[1]
    batchsize = trans.size()[0]
    I = torch.eye(d)[None, :, :]
    if trans.is_cuda:
        I = I.to(trans.device)
    loss = torch.mean(torch.norm(torch.bmm(trans, trans.transpose(2,1)) - I, dim=(1,2)))
    return loss

class PointNetCls(nn.Module):
    def __init__(self, k=2, feature_transform=False):
        super(PointNetCls, self).__init__()
        self.feature_transform = feature_transform
        self.feat = PointNetfeat(global_feat=True, feature_transform=feature_transform)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k)
        self.dropout = nn.Dropout(p=0.3)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.dropout(self.fc2(x))))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1), trans, trans_feat

In [ ]:
tensor = torch.randn(32,1000,3)
permuted_tensor = tensor.permute(0,2,1)
transposed_tensor = tensor.transpose(2, 1)

print("Oryginalny tensor ", tensor.shape)
print("Permutacja ", permuted_tensor.shape)
print("Transpozycja ", transposed_tensor.shape)

lin1 = nn.Linear(3, 64)           #DEF
conv1 = nn.Conv1d(3, 64, 1)       #DEF

lin1.weight = torch.nn.Parameter(conv1.weight.squeeze(2))
lin1.bias = torch.nn.Parameter(conv1.bias)

out_linear = lin1(tensor).transpose(2, 1)
out_conv = conv1(permuted_tensor)

print("Pierwsza warstwa liniowa (mean) ", out_linear.mean())
print("Pierwsza warstwa liniowa (shape) ", out_linear.shape)
print("Pierwsza warstwa konwolucyjna (mean) ", out_conv.mean())
print("Pierwsza warstwa konwolucyjna (shape) ", out_conv.shape)

bn1 = nn.BatchNorm1d(64)          #DEF

out_bnlinear = bn1(out_linear)
out_bnconv = bn1(out_conv)

print("BatchNorm64 - liniowa", out_bnlinear.shape)
print("BatchNorm64 - konwolucyjna", out_bnconv.shape)

out_linearrelu = F.relu(out_bnlinear)
out_convrelu = F.relu(out_bnconv)

print("ReLU - liniowa", out_linearrelu.shape)
print("ReLU - konwolucyjna", out_convrelu.shape)

lin2 = nn.Linear(64, 128)         #DEF
conv2 = nn.Conv1d(64, 128, 1)     #DEF

out_2linear = lin2(out_linearrelu.transpose(2, 1)).transpose(2,1)
out_2conv = conv2(out_convrelu)

print("2 warstwa - liniowa", out_2linear.shape)
print("2 warstwa - konwolucyjna", out_2conv.shape)

bn2 = nn.BatchNorm1d(128)          #DEF

out_2bnlinear = bn2(out_2linear)
out_2bnconv = bn2(out_2conv)

Oryginalny tensor  torch.Size([32, 1000, 3])
Permutacja  torch.Size([32, 3, 1000])
Transpozycja  torch.Size([32, 3, 1000])
Pierwsza warstwa liniowa (mean)  tensor(-0.0141, grad_fn=<MeanBackward0>)
Pierwsza warstwa liniowa (shape)  torch.Size([32, 64, 1000])
Pierwsza warstwa konwolucyjna (mean)  tensor(-0.0141, grad_fn=<MeanBackward0>)
Pierwsza warstwa konwolucyjna (shape)  torch.Size([32, 64, 1000])
BatchNorm64 - liniowa torch.Size([32, 64, 1000])
BatchNorm64 - konwolucyjna torch.Size([32, 64, 1000])
ReLU - liniowa torch.Size([32, 64, 1000])
ReLU - konwolucyjna torch.Size([32, 64, 1000])
2 warstwa - liniowa torch.Size([32, 128, 1000])
2 warstwa - konwolucyjna torch.Size([32, 128, 1000])


#TRENOWANIE (WŁASNA IMPLEMENTACJA)

In [ ]:
confOptim = OmegaConf.load('optimizer.yaml')
confSched = OmegaConf.load('scheduler.yaml')

optionsOptim = instantiate(confOptim)
optionsSched = instantiate(confSched)

blue = lambda x: '\033[94m' + x + '\033[0m'

options.manualSeed = random.randint(1, 10000)  # fix seed
print("Random Seed: ", options.manualSeed)
random.seed(options.manualSeed)
torch.manual_seed(options.manualSeed)

dataset = ModelNetLoader(root=dataset_root, num_classes=dataset_num_classes, batch_size=options.batch_size, split='train', numpoints=options.num_points)
testdataset = ModelNetLoader(root=dataset_root, num_classes=dataset_num_classes, batch_size=options.batch_size, split='test', numpoints=options.num_points)

traindataset_size = int(len(dataset) * 0.9)
traindataset, valdataset = random_split(dataset, [traindataset_size, len(dataset) - traindataset_size])

print("Rozmiar zbioru treningowego: " + str(len(traindataset)))
print("Rozmiar zbioru walidacyjnego: " + str(len(valdataset)))
print("Rozmiar zbioru testowego: " + str(len(testdataset)))
print("Rozmiar batcha: " + str(dataset.batch_size))
print('Liczba klas: ' + str(dataset.num_classes))

traindataloader = DataLoader(traindataset, batch_size=options.batch_size, num_workers=int(options.num_workers), shuffle=True)
valdataloader = DataLoader(valdataset, batch_size=len(valdataset), num_workers=int(options.num_workers), shuffle=True)
testdataloader = DataLoader(testdataset, batch_size=options.batch_size, num_workers=int(options.num_workers), shuffle=False)

classifier = PointNetCls(k=dataset.num_classes, feature_transform=options.feature_transform)

num_batch = len(traindataset) / options.batch_size

try:
    os.makedirs(options.out_folder)
except OSError:
    pass

# CHECKPOINT MODELU UWARUNKOWANY PLIKIEM classification.yaml
if options.model != '':
    classifier.load_state_dict(torch.load(options.model))
    options.num_epochs = 0

optimizer = optim.Adam(classifier.parameters(), confOptim.lr, confOptim.betas)
scheduler = optim.lr_scheduler.StepLR(optimizer, confSched.step_size, confSched.gamma)

#classifier.to('cuda:0')

if options.single_batch_overfit:
    single_batch_overfit = torch.empty((options.batch_size, options.num_points, 3))
    single_batch_overfit_cls = torch.empty((options.batch_size))
    for n in range(options.batch_size):
        item = dataset[100*n]
        single_batch_overfit[n, :, :] = item[0]
        single_batch_overfit_cls[n] = item[1]
    single_batch_overfit_cls = single_batch_overfit_cls.type(torch.int64)

writer = SummaryWriter()#tensorboard
y_pred=[]
y_true=[]
classes = tuple([x for x in range(40)])
i=0


for epoch in range(options.num_epochs):
    scheduler.step()
    for i, data_ in enumerate(traindataloader, 0): 
        if not options.single_batch_overfit:
          points, target = data_
        else:
          points, target = single_batch_overfit, single_batch_overfit_cls
        #points = points.transpose(2, 1)
        #points, target = points.to('cuda:0'), target.to('cuda:0')
        optimizer.zero_grad()
        classifier = classifier.train()
        pred, trans, trans_feat = classifier(points)
        loss = F.nll_loss(pred, target)
        if options.feature_transform:
            loss += feature_transform_regularizer(trans_feat) * 0.001
        loss.backward()
        optimizer.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.data).cpu().sum()
        print('Epoch %d: batch %d of %d. Train loss: %f Accuracy: %f (%i/%i)' % (epoch+1, i, num_batch, loss.item(), correct.item() / float(options.batch_size), int(correct.item()), int(options.batch_size)))
    j, data_ = next(enumerate(valdataloader, 0))
    if not options.single_batch_overfit:
      points, target = data_
    else:
      points, target = single_batch_overfit, single_batch_overfit_cls
    #points = points.transpose(2, 1)
    #points, target = points.to('cuda:0'), target.to('cuda:0')
    classifier = classifier.eval()
    pred, _, _ = classifier(points)
    loss = F.nll_loss(pred, target)
    pred_choice = pred.data.max(1)[1]
    correct = pred_choice.eq(target.data).cpu().sum()
    print('Epoch %d: batch %d of %d. %s loss: %f Accuracy: %f' % (epoch+1, i, num_batch, blue('Validation'), loss.item(), correct.item()/float(len(valdataset))))
    torch.save(classifier.state_dict(), '%s/cls_model_%d.pth' % (options.out_folder, epoch))
    #tensorboard 
    y_pred.extend( (torch.max(torch.exp(pred), 1)[1]).data.cpu().numpy() )
    y_true.extend( target.data.cpu().numpy() )
    cf_matrix = confusion_matrix(y_true, y_pred,labels=classes)
    df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) * 10,index=[i for i in classes], columns=[i for i in classes]) 
    writer.add_figure("Confusion matrix", sn.heatmap(df_cm, annot=True).get_figure(), epoch)
    writer.add_scalar('loss', loss.item(), epoch) 
    writer.add_scalar('accuracy', correct.item()/float(len(valdataset)), epoch)    
    #


total_correct = 0
total_testset = 0
print(blue("Testing"))

for i,data_ in tqdm(enumerate(testdataloader, 0)):
    if not options.single_batch_overfit:
      points, target = data_
    else:
      points, target = single_batch_overfit, single_batch_overfit_cls
    points = points.transpose(2, 1)
    points, target = points.to('cuda:0'), target.to('cuda:0')
    classifier = classifier.eval()
    pred, _, _ = classifier(points)
    pred_choice = pred.data.max(1)[1]
    correct = pred_choice.eq(target.data).cpu().sum()
    total_correct += correct.item()
    total_testset += points.size()[0]

print("\nFinal accuracy: {}".format(total_correct / float(total_testset)))

Random Seed:  7990
Rozmiar zbioru treningowego: 8858
Rozmiar zbioru walidacyjnego: 985
Rozmiar zbioru testowego: 2468
Rozmiar batcha: 32
Liczba klas: 40


RuntimeError: ignored

#TENSORBOARD

In [ ]:
%load_ext tensorboard

In [ ]:
!kill 2278

In [ ]:
%tensorboard --logdir runs